In [2]:
## fill nas with median
## very rude transformation of dates
## test/train split
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
from utils import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn import metrics
# from fancyimpute import *

%matplotlib inline

In [29]:
from fastai.imports import *
from fastai.structured import *
#from pandas_summary import DataFrameSummary

In [3]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

In [62]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype, low_memory=True)
test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype, low_memory=True)

In [5]:
test['is_train'] = 0
data['is_train'] = 1

In [6]:
df_combine = pd.concat([data, test], axis=0, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [7]:
df_combine = df_combine.drop('label', axis =1).fillna(-1)

In [8]:
y = df_combine['is_train'].values #labels
X = df_combine.drop(['is_train','id','date'], axis=1).values #covariates or our independent variables

In [9]:
m = RandomForestClassifier(n_jobs=-1, max_depth=5, min_samples_leaf = 5, max_features=100)
predictions = np.zeros(y.shape) #creating an empty prediction array

In [10]:
skf = SKF(n_splits=10, shuffle=True, random_state=100)
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    m.fit(X_train, y_train)
    probs = m.predict_proba(X_test)[:, 1] #calculating the probability
    predictions[test_idx] = probs

In [11]:
fpr, tpr, thresholds = metrics.roc_curve(y, predictions)

In [12]:
print('ROC-AUC for train and test distributions:', metrics.auc(fpr,tpr))

ROC-AUC for train and test distributions: 0.953422704860055


In [14]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [32]:
save_obj(m, 'train_test_shift')

In [58]:
test = load_obj('train_test_shift')

In [47]:
order = [(name, importance) for name, importance in zip(data.columns[3:], m.feature_importances_)]

In [59]:
[(x, _) for _,x in sorted(zip(test.feature_importances_,data.columns[3:]), reverse=True)]

[('f243', 0.5151336868205257),
 ('f135', 0.08162090423187165),
 ('f259', 0.04865402075230311),
 ('f113', 0.04755930555690895),
 ('f36', 0.043068999249312054),
 ('f6', 0.03285556942379818),
 ('f280', 0.024778265376039433),
 ('f216', 0.02464259766571218),
 ('f218', 0.020056105392005086),
 ('f55', 0.02004756042558175),
 ('f5', 0.012578410048515246),
 ('f217', 0.012528895507138996),
 ('f215', 0.011712389713565328),
 ('f155', 0.011613156956922856),
 ('f224', 0.00956908031701983),
 ('f202', 0.008317787450127501),
 ('f124', 0.007997170183093849),
 ('f8', 0.007766336859191118),
 ('f146', 0.00649983593872751),
 ('f225', 0.006280709436529632),
 ('f226', 0.0061706619130781955),
 ('f213', 0.006054562660154618),
 ('f204', 0.005787413778072243),
 ('f206', 0.005630852984597675),
 ('f261', 0.003650923070535668),
 ('f151', 0.0029100884512361105),
 ('f154', 0.002417448921660642),
 ('f56', 0.0018626657780537307),
 ('f265', 0.0017718835274788901),
 ('f244', 0.0016594423042168584),
 ('f198', 0.001012276482

In [60]:
data.f243.unique()

array([  3.,   2.,   1.,  15., 203., 169.,   7.,  43.,  36.,  45., 118., 120., 195., 197., 150., 179., 177.,
       198.,   4., 189., 127., 158., 165., 171., 191.,  13.,  22., 111., 201.,   8., 149.,  24.,  10., 192.,
       206., 174., 196., 164.,  16., 184., 156., 172., 168., 182.,  47., 194., 175.,  80., 137.,  21.,  34.,
       190.,  25., 187.,  30.,  55.,   6., 173., 193., 188.,  18.,  39., 186.,  27.,  17.,  11.,  81.,  51.,
       199., 157.,  65., 140.,  12., 178.,  71., 183., 180., 161., 200.,  20.,   9., 181.,   5.,  62.,  23.,
       163.,  28., 176., 185.,  82., 102.,  35.,  64., 216., 146.,  46., 160.,  19.,  59.,  33.,  14.,  38.,
        nan, 217.,  91.,  42., 130., 135.,  31.,  37.,  75.,  49., 147., 214., 207., 205., 125.,  70., 159.,
        41., 162.,  68., 166., 139., 104., 145.,  61.,  77., 144., 151., 170.,  98.,  32.,  63.,  74., 131.,
        29., 133., 110.,  73.,  78., 154., 136., 202., 212.,  50., 204., 230.,  66., 167., 126., 134., 128.,
        86., 153., 

In [63]:
test.f243.unique()

array([  4.,   2.,   1., 172., 192., 188.,  13.,   3., 182., 189.,   5.,  23., 179., 186.,  75., 133., 199.,
        41., 193., 223.,  28.,  24., 104., 190., 178.,   7., 195., 201.,   8., 191., 196., 181., 281.,  17.,
       177., 180., 279.,  nan,  19.,   6., 184.,  14., 269., 144.,  94., 173.,  10.,   9., 200., 143.,  35.,
       206., 183., 202., 167., 209., 161.,  43.,  30.,  53., 185., 194., 175., 205.,  52., 197., 187., 215.,
        26., 110., 213.,  22.,  91.,  15., 176., 263., 272., 203., 212., 198., 211., 111., 276., 114.,  90.,
        47., 260.,  38.,  44.,  36.,  29., 160., 230.,  67.,  66., 222., 171., 219.,  72.,  45.,  12.,  18.,
        21., 210., 214., 266., 217., 174., 265.,  37.,  54.,  27.,  39., 273.,  51., 271.,  11., 168., 131.,
        20.,  16., 115.,  50., 261.,  92., 141., 166.,  56., 264.,  81., 204.,  74., 138., 274.,  55., 225.,
       113., 275.,  31., 278., 216., 227.,  73., 267.,  93.,  32.,  25.,  88.,  80., 255., 107., 224., 109.,
        46.,  42., 